Install OpenAI Agents Dep.

In [ ]:
!pip install -Uq openai-agents

In [ ]:
import nest_asyncio
nest_asyncio.apply()

Provider Config

In [ ]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")
     

#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemini-2.0-flash-lite-preview-02-05:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free
     


In [ ]:
Using the OpenRouter API directly

In [ ]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

In [ ]:
data = response.json()
data['choices'][0]['message']['content']

2. Using OpenAI Agents SDK

In [2]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'openai'

In [ ]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, RunConfig

load_dotenv()
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

# Check If Key Exits
if not openrouter_api_key:
    raise ValueError("OPENROUTER_API_KEY Not Set. Please Ensure Defined Your .env file")

# Setup OpenRouter Client ( Like OpenAI , But Via OpenRouter )
external_client = AsyncOpenAI(
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)

# Chose Any OpenRouter-Supported Model
model = OpenAIChatCompletionsModel(
    model="opengvlab/internvl3-14b:free",
    openai_client=external_client
)

# Steup Config
config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

agent = Agent(
    name = "Chief - X2 Writer Agent",
    instructions = "You Are A Writer Agent. Generate Stories, Novels, Urdu Spy Series ETC."
)

response = Runner.run_sync(
    agent,
    input = "Write A Short Story Introduction Imran Series SPY Novel In Urdu & English",
    run_config = config
)

print(response)

ModuleNotFoundError: No module named 'dotenv'